In [1]:
# |0> - U3(theta, phi, lambda) - |Psi(theta, phi, lambda)> = U3(theta, phi, lambda)|0>

# We want multiple measurements of |psi(theta, phi, lambda)> in the Z-basis
# to match a predefined probability distribution (target distribution): 
# p_target = [x_target, y_target], with x_target + y_target = 1

# We use the COBYLA optimizer 
# We measure our error with the Manhattan distance: d(A,B) = |x_target - x_output| + |y_target - y_output|

In [2]:
# We first create the random probability vector in python:

import numpy as np

np.random.seed(999999)

target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

print(target_distr)

[0.51357006 0.48642994]


In [3]:
# We subsequently create a function that takes the parameters of our single U3 variational form 
# as arguments and returns the corresponding quantum circuit:

from qiskit import *

def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [4]:
# Now we specify the objective function which takes as input a list of the variational 
# form's parameters, and returns the cost associated with those parameters:

backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    job = execute(qc, backend=backend, shots=NUM_SHOTS)
    result = job.result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

In [5]:
# Finally, we create an instance of the COBYLA optimizer, and run the algorithm:

from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0]) 
job = execute(qc, backend=backend, shots=NUM_SHOTS)
counts = job.result().get_counts()
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5279, 0.4721]
Output Error (Manhattan Distance): 0.017259881261160848
Parameters Found: [1.51953868 1.49204907 1.76658905]
